In [81]:
import os
import requests
import pandas as pd
from openai import OpenAI

from dotenv import load_dotenv

load_dotenv()

True

In [82]:
def refresh_access_token():
    res = requests.post(
        "https://www.strava.com/oauth/token",
        data={
            "client_id": os.getenv("STRAVA_CLIENT_ID"),
            "client_secret": os.getenv("STRAVA_CLIENT_SECRET"),
            "refresh_token": os.getenv("STRAVA_REFRESH_TOKEN"),
            "grant_type": "refresh_token",
        }
    )
    print(res.status_code)
    print(res.json())
    res.raise_for_status()
    return res.json()["access_token"]

In [83]:
def get_strava_activities(access_token, per_page=5):
    url = "https://www.strava.com/api/v3/athlete/activities"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {"per_page": per_page}
    
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()

In [ ]:
# get access token
token_data = refresh_access_token()
# call up strava
strava_data = get_strava_activities(access_token=token_data, per_page=10)


200
{'token_type': 'Bearer', 'access_token': '51e877d88b10b5f79b5a1f4c763dd44e4a3bdd24', 'expires_at': 1748647824, 'expires_in': 21600, 'refresh_token': 'c280cb6536c00d9cbcd05b660423cb5ee91e34c8'}


In [86]:
strava_results = pd.DataFrame(strava_data)

In [109]:
strava_results['moving_time_mins'] = round(strava_results['moving_time']/60, 1)
strava_results['distance_miles'] = round(strava_results['distance']/1609.34, 2)
strava_results['elevation_gain_ft'] = round(strava_results['total_elevation_gain']*3.281, 1)


In [110]:
filtered_strava_results = strava_results[['distance_miles', 'moving_time_mins', 'elevation_gain_ft', 'type', 'start_date_local', 'average_speed', 'average_heartrate', 'max_heartrate']]

In [111]:
filtered_strava_results

,distance_miles,moving_time_mins,elevation_gain_ft,type,start_date_local,average_speed,average_heartrate,max_heartrate
0,3.02,40.5,112.9,Run,2025-05-28T17:35:23Z,1.995,145.0,158.0
1,8.53,97.3,369.4,Run,2025-05-26T11:52:59Z,2.352,169.3,187.0
2,4.99,55.8,0.0,Run,2025-05-23T09:14:03Z,2.394,161.9,180.0
3,6.01,68.7,168.0,Run,2025-05-20T07:30:00Z,2.347,163.2,180.0
4,4.12,92.2,874.1,Walk,2025-05-19T19:45:26Z,1.200,102.8,161.0
5,5.50,73.8,232.3,Run,2025-05-18T17:34:04Z,1.999,149.7,170.0
6,4.51,52.8,34.8,Run,2025-05-16T10:10:11Z,2.293,174.4,192.0
7,2.55,29.1,207.0,Run,2025-05-13T12:47:05Z,2.354,163.2,182.0
8,6.04,69.5,207.0,Run,2025-05-11T12:53:14Z,2.334,169.3,189.0
9,3.54,42.0,143.4,Run,2025-05-06T09:55:27Z,2.263,172.2,188.0


In [40]:
filtered_results['distance_miles'] = filtered_results['distance']/1609.34

/var/folders/dv/26p0y0154lq0348hysv389ym0000gp/T/ipykernel_17339/1696999570.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_results['distance_miles'] = filtered_results['distance']/1609.34


In [41]:
filtered_results

,distance,moving_time,elapsed_time,total_elevation_gain,type,start_date_local,average_speed,average_heartrate,max_heartrate,distance_miles
0,4852.7,2432,3015,34.4,Run,2025-05-28T17:35:23Z,1.995,145.0,158.0,3.015335
1,13727.9,5836,6398,112.6,Run,2025-05-26T11:52:59Z,2.352,169.3,187.0,8.530143
2,8022.9,3351,3765,0.0,Run,2025-05-23T09:14:03Z,2.394,161.9,180.0,4.985211
3,9672.0,4121,4732,51.2,Run,2025-05-20T07:30:00Z,2.347,163.2,180.0,6.009917
4,6638.5,5534,5588,266.4,Walk,2025-05-19T19:45:26Z,1.200,102.8,161.0,4.124983
5,8858.7,4431,4889,70.8,Run,2025-05-18T17:34:04Z,1.999,149.7,170.0,5.504555
6,7260.7,3167,3739,10.6,Run,2025-05-16T10:10:11Z,2.293,174.4,192.0,4.511601
7,4109.8,1746,2066,63.1,Run,2025-05-13T12:47:05Z,2.354,163.2,182.0,2.553718
8,9728.3,4168,4803,63.1,Run,2025-05-11T12:53:14Z,2.334,169.3,189.0,6.044900
9,5702.5,2520,2848,43.7,Run,2025-05-06T09:55:27Z,2.263,172.2,188.0,3.543378


In [ ]:
client = OpenAI()

# NOTES ON PROMPT ENGINEERING
# -- If you say it's an expert on history of SF as well as local flora/fauna and then ask for a fact, every fact will be about GGP being bigger than central park
# -- chatGPT seems to hallucinate a lot more via API than in the GUI even for the exact same model/prompt.

response = client.responses.create(
    model="gpt-4.1",
    # tools=[{"type": "web_search_preview"}],
    instructions="""You are a supportive life coach and personal trainer named Zeus Groos. You are also knowledgeable
        about the history of San Francisco as well as local flora and fauna. You do not make up any information and only say 
        things that you know to be true. Your trainee, Zach, is training for the New York Marathon on November 3 2025.
        He is doing a half marathon training program that ends on June 29 and then he'll begin the full marathon training program.
        This is his first competetive marathon and his target time is 4 hours.
        """,
    input="""Greet me with a motivational message, tell me the date (for PST time zone), and tell me an interesting fact please! Use the internet to check the date and fact accuracy and do not hallucinate anything.
        example fact themes:
        - historical anecdotes from san francisco, especially if they occurred on today's date
        - facts about native flora or fauna of san francisco""",
    # temperature=0.9
)

print(response.output_text)


Good morning, Zach! Today is **Sunday, June 9, 2024** (PST). Every training day is a brick in the foundation of your marathon success—keep stacking them, and your dream finish line will become reality!

Here’s an interesting San Francisco historical tidbit for you: On June 9, 1968, the last regularly scheduled cable car ran on the O’Farrell, Jones and Hyde line in San Francisco. The city—home of the world’s last manually operated cable car system—preserved this unique heritage, and the cable cars still run today on three main lines, serving as both transport and moving history.

Stay strong and curious—both on your runs and about the world around you!
